# **Setting Up**

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install gradio

In [ ]:
# Connect to Google Drive for files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive"

In [ ]:
# Important Stuff
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
import os
import gradio as gr
os.environ["OPENAI_API_KEY"] = ""

# **Reading the PDF**

In [ ]:
reader = PdfReader("/test/AS.pdf")

In [ ]:
# Read Data from Text:
raw_text = ''

# **CharacterTextSplitter**

In [ ]:
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
# Split Text Data:
text_splitter = CharacterTextSplitter(
    separator= "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

In [ ]:
texts = text_splitter.split_text(raw_text)

# **Embeddings**

In [ ]:
embeddings = OpenAIEmbeddings()

# **FAISS - Facebook A.I. Similarity Search**

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

##**LangChain**

In [ ]:
# Using ChatGPT-3.5-turbo for the document answering
llm = ChatOpenAI(temperature=0.0)

**Chain**

In [ ]:
# Question & Answer Chain
chain = load_qa_chain(llm = llm, chain_type="stuff")

# **Creating Function for our Questions**

In [ ]:
def ask_GPT(question):
    query = question
    docs = docsearch.similarity_search(query)
    response = chain.run(input_documents = docs, question=query)
    return response

In [ ]:
# Ask Question Here!
ask_GPT("")

# **Web Interface**

In [ ]:
demo = gr.Interface(
    fn = ask_GPT,
    inputs = gr.Textbox(lines = 2, placeholder="Enter your prompt: "),
    outputs = "text"
)
demo.launch(share = True)